In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Загрузка предобученной модели
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [ ]:
import pandas as pd
df = pd.read_csv('data/labeled.csv')
df['toxic'].dtype

dtype('float64')

In [51]:
# Преобразование столбца float_column в int
df["toxic"] = df["toxic"].astype(int)

In [52]:
import torch
# Преобразуем тексты в формат, который понимает модель
def encode_data(texts, tokenizer, max_length=512):
    return tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")



In [53]:
# Создаем данные для теста (например, 20% от общего датасета)
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["comment"].tolist(), df["toxic"].tolist(), test_size=0.2, random_state=42
)

# Преобразование данных
train_encodings = encode_data(train_texts, tokenizer)
train_labels = torch.tensor(train_labels)
# Кодируем данные для теста
val_encodings = encode_data(val_texts, tokenizer)
val_labels = torch.Tensor(val_labels)




In [54]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

# Класс Dataset для совместимости с Trainer
class ToxicityDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["toxic"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Создаем dataset
train_dataset = ToxicityDataset(train_encodings, train_labels)
val_dataset = ToxicityDataset(val_encodings, torch.tensor(val_labels))



/var/folders/2d/c6h6_91n28vgt5yh6y3kjjx40000gn/T/ipykernel_27748/336227031.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = ToxicityDataset(val_encodings, torch.tensor(val_labels))


In [55]:
from transformers import Trainer
import torch.nn as nn
import torch

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Переопределяем метод compute_loss для использования кастомной функции потерь.
        """
        labels = inputs.pop("toxic")  # Извлекаем метки
        outputs = model(**inputs)  # Пропускаем данные через модель
        logits = outputs.logits  # Получаем логиты
        
        # Используем Binary Cross-Entropy Loss
        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(logits.view(-1), labels.float())
        
        return (loss, outputs) if return_outputs else loss

In [56]:
# Параметры тренировки
training_args = TrainingArguments(
    output_dir="./results",          # Папка для сохранения модели
    evaluation_strategy="epoch",     # Оценка после каждой эпохи
    learning_rate=2e-4,              # Скорость обучения
    per_device_train_batch_size=32,   # Размер батча
    num_train_epochs=7,              # Количество эпох
    weight_decay=0.01,               # Регуляризация
    logging_dir="./logs",            # Логи
    save_strategy="epoch",           # Сохранение модели после каждой эпохи
    no_cuda=False,                   # Если работаете с CUDA или MPS
)

trainer = CustomTrainer(
    model=model,                        # Модель
    args=training_args,                 # Параметры обучения
    train_dataset=train_dataset,        # Тренировочный датасет
    eval_dataset=val_dataset            # Оценочный датасет
)

# Начинаем обучение
trainer.train()

/opt/anaconda3/envs/gey/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/2d/c6h6_91n28vgt5yh6y3kjjx40000gn/T/ipykernel_27748/336227031.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


KeyError: 'toxic'